In [156]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

In [157]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-01-01")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-01-01")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)

C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_17656\4222972167.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_17656\4222972167.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed
C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_17656\4222972167.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-01-01")
[*********************100%***********************]  1 of 1 completed


In [158]:
df_apple.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,72.53851318359375,72.59889150198023,71.29230380764884,71.54588979671196,135480400
3,2020-01-03,71.83330535888672,72.594070846621,71.6086997390569,71.76568192376371,146322800
4,2020-01-06,72.40567016601562,72.4443131708714,70.70300468370534,70.95418053006813,118387200


In [159]:
df_microsoft.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,153.0422821044922,153.1470931177711,150.86032587757265,151.28909291774593,22622100
3,2020-01-03,151.13665771484375,152.4039130274651,150.60307959140601,150.8508226765117,21116200
4,2020-01-06,151.52734375,151.59404829608147,149.12622725263657,149.66934299444563,20813700


In [160]:
df_amazon.head()

,Price,Close,High,Low,Open,Volume
0,Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
1,Date,NaN,NaN,NaN,NaN,NaN
2,2020-01-02,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000
3,2020-01-03,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000
4,2020-01-06,95.14399719238281,95.18450164794922,93.0,93.0,81236000


In [161]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

In [162]:
df_apple.head()

,Close,High,Low,Open,Volume,Date
2,72.53851318359375,72.59889150198023,71.29230380764884,71.54588979671196,135480400,2020-01-02
3,71.83330535888672,72.594070846621,71.6086997390569,71.76568192376371,146322800,2020-01-03
4,72.40567016601562,72.4443131708714,70.70300468370534,70.95418053006813,118387200,2020-01-06
5,72.0651626586914,72.67135607636844,71.84538465436648,72.41535254834196,108872000,2020-01-07
6,73.22441864013672,73.52631026770732,71.76809372355218,71.76809372355218,132079200,2020-01-08


In [163]:
df_microsoft.head()

,Close,High,Low,Open,Volume,Date
2,153.0422821044922,153.1470931177711,150.86032587757265,151.28909291774593,22622100,2020-01-02
3,151.13665771484375,152.4039130274651,150.60307959140601,150.8508226765117,21116200,2020-01-03
4,151.52734375,151.59404829608147,149.12622725263657,149.66934299444563,20813700,2020-01-06
5,150.145751953125,152.13714723225632,149.89802336904734,151.8036681235651,21634100,2020-01-07
6,152.5373077392578,153.21381792157746,150.49826874289445,151.43203046001807,27746500,2020-01-08


In [164]:
df_amazon.head()

,Close,High,Low,Open,Volume,Date
2,94.90049743652344,94.90049743652344,93.2074966430664,93.75,80580000,2020-01-02
3,93.74849700927734,94.30999755859375,93.2249984741211,93.2249984741211,75288000,2020-01-03
4,95.14399719238281,95.18450164794922,93.0,93.0,81236000,2020-01-06
5,95.34300231933594,95.69450378417969,94.60199737548828,95.2249984741211,80898000,2020-01-07
6,94.59850311279297,95.55000305175781,94.3219985961914,94.9020004272461,70160000,2020-01-08


In [165]:
df_apple.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 2 to 1259
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   1258 non-null   object
 1   High    1258 non-null   object
 2   Low     1258 non-null   object
 3   Open    1258 non-null   object
 4   Volume  1258 non-null   object
 5   Date    1258 non-null   object
dtypes: object(6)
memory usage: 59.1+ KB


In [166]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1258,1258,1258,1258,1258,1258
unique,1243,1258,1258,1258,1256,1258
top,139.26515197753906,72.59889150198023,71.29230380764884,71.54588979671196,90956700,2020-01-02
freq,2,1,1,1,2,1


In [167]:
df_apple.isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
Date      0
dtype: int64

In [168]:
df_apple['Close'] = df_apple['Close'].astype(float)
df_apple['High'] = df_apple['High'].astype(float)
df_apple['Low'] = df_apple['Low'].astype(float)
df_apple['Open'] = df_apple['Open'].astype(float)
df_apple['Volume'] = df_apple['Volume'].astype(float)
df_apple['Date'] = pd.to_datetime(df_apple['Date'])

In [169]:
df_apple.describe()

,Close,High,Low,Open,Volume,Date
count,1258.000000,1258.000000,1258.000000,1258.000000,1.258000e+03,1258
mean,151.806048,153.332133,150.104333,151.651295,9.057103e+07,2022-07-01 15:39:46.645468928
min,54.316936,55.316755,51.470001,55.215079,2.323470e+07,2020-01-02 00:00:00
25%,126.636354,127.757510,124.630113,126.080069,5.546825e+07,2021-04-02 00:00:00
50%,150.392868,152.138959,148.455981,150.212647,7.628335e+07,2022-06-30 12:00:00
75%,176.399673,177.954445,174.830449,176.205319,1.077425e+08,2023-10-01 06:00:00
max,258.103729,259.179926,256.718662,257.276679,4.265100e+08,2024-12-31 00:00:00
std,41.970402,42.111163,41.747076,41.914049,5.324438e+07,NaN


In [170]:
df_apple[df_apple['Close'] < 0].head()

,Close,High,Low,Open,Volume,Date


In [171]:
#AAPL FEATURE ENGINEERING
window = 14

df_apple['Price Change'] = df_apple['Close'].diff().fillna(0)

df_apple['Gain'] = df_apple.loc[df_apple['Price Change'] > 0, 'Price Change']
df_apple['Loss'] = -df_apple.loc[df_apple['Price Change'] < 0, 'Price Change']

df_apple['Gain'] = df_apple['Gain'].fillna(0)
df_apple['Loss'] = df_apple['Loss'].fillna(0)

df_apple['Average Gain'] = df_apple['Gain'].rolling(window=window).mean().fillna(0)
df_apple['Average Loss'] = df_apple['Loss'].rolling(window=window).mean().fillna(0)

df_apple = df_apple.drop(index=list(range(2,15)))

df_apple['RS'] = df_apple['Average Gain'] / df_apple['Average Loss']

df_apple['RSI'] = 100 - (100 / (1 + df_apple['RS']))

df_apple['Target Close'] = df_apple['Close'].shift(-1)

df_apple.head()
# df_apple['Average Gain']
# df_apple['Average Loss']
# df_apple['RS']
# df_apple['RSI']

,Close,High,Low,Open,Volume,Date,Price Change,Gain,Loss,Average Gain,Average Loss,RS,RSI,Target Close
15,76.728783,77.281843,76.634589,76.941308,101832400.0,2020-01-22,0.272903,0.272903,0.000000,0.508211,0.208906,2.432722,70.868603,77.098274
16,77.098274,77.177970,76.233653,76.781892,104472000.0,2020-01-23,0.369492,0.369492,0.000000,0.534604,0.208906,2.559058,71.902676,76.876091
17,76.876091,78.088485,76.685293,77.344627,146537600.0,2020-01-24,-0.222183,0.000000,0.222183,0.534604,0.174405,3.065306,75.401603,74.615540
18,74.615540,75.296601,73.632579,74.883616,161940000.0,2020-01-27,-2.260551,0.000000,2.260551,0.493720,0.335873,1.469963,59.513568,76.726341
19,76.726341,76.897814,75.398018,75.497040,162234000.0,2020-01-28,2.110802,2.110802,0.000000,0.644492,0.311551,2.068659,67.412472,78.332413
